In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

## 5.1 - 합성곱 신경망 소개
###### Python 3.5.5. keras 2.1.6에서 실행 확인하였습니다.
책의 5장 1절의 코드 예제입니다. 아래 모델에에 대해서 기술하세요.



### 코드 5-1 간단한 컨브넷 만들기
- 컨브넷은 (image_height, image_width, image_channels)  크기의 입력 텐서를 사용함.
- 첫 번째 합성곱 층이 (28, 28, 1) 크기의 특성 맵을 입력값으로 받아 Conv2D(32,(3,3) 만큼 합성곱 연산 출력된 응답 맵은 (26,26,32) -> 높이 너비가 2만큼 줄어든 이유는 경계 문제 때문 Padding을 매개변수('same' = 동일한 이미지 사이즈, 'valid' = 사용하지 않음)로 사용하여 동일한 높이와 넓이를 가진 응답맵(=출력 특성맵)을 얻을 수 있음
- 출력된 특성맵을 다운샘플링 하기 위하여 맥스풀링 연산 -> MaxPolling2D((2,2))연산을 통해 출력맵의 높이와 너비가 반씩 줄어듬.
- 3번의 합성곱 연산과 2번의 맥스풀링 연산을통해 (3,3,64) 크기를 가진 특성맵을 얻음.



In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3),padding = 'valid', activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3),padding = 'valid', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3),padding = 'valid', activation='relu'))
model.summary()

W0811 13:02:12.189779 15860 deprecation_wrapper.py:119] From C:\Users\lymmo\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 13:02:12.210724 15860 deprecation_wrapper.py:119] From C:\Users\lymmo\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 13:02:12.212728 15860 deprecation_wrapper.py:119] From C:\Users\lymmo\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 13:02:12.228712 15860 deprecation_wrapper.py:119] From C:\Users\lymmo\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


### 코드 5-2 컨브넷 위에 분류기 추가하기
- 전결합층에 전달 하기 위하여 합성곱 연산을 완료한 특성맵을 Flatten Layer를 통해 1차열 배열로 바꾸어 준다.(Flatten결과 = (576, ))
- Dense레이어를 통해 입출력을 모두 연결해줌 
- 10개의 클래스를 분류하기 위해서 출력 크기를 10으로 지정. 소프트맥스 활성화 함수를 사용(Softmax는 입력받은 값을 출력으로 0~1사이의 값으로 모두 정규화 하는것을 말함.)

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

### 코드 5-3 이미지에 컨브넷 훈련하기
- 훈련세트(train_images, train_labels)를 구성, 테스트 세트(test_images, test_labels)에서 테스트.
- 훈련 데이터와 테스트 데이터를 0과 1사이 값을 가지는 float32 타입으로 바꾸어줌.
- 에포크 5만큼 데이터를 학습.

In [4]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [5]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

W0811 13:02:12.806203 15860 deprecation_wrapper.py:119] From C:\Users\lymmo\AppData\Roaming\Python\Python37\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0811 13:02:12.824116 15860 deprecation_wrapper.py:119] From C:\Users\lymmo\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0811 13:02:12.908857 15860 deprecation.py:323] From C:\Users\lymmo\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0811 13:02:13.001609 15860 deprecation_wrapper.py:119] From C:\Users\lymmo\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_back

Epoch 1/5
60000/60000 [==============================] - 31s 512us/step - loss: 0.1777 - acc: 0.9448
Epoch 2/5
60000/60000 [==============================] - ETA: 0s - loss: 0.0493 - acc: 0.9848- ETA: 0s - loss: 0.0493 - acc: 0. - 31s 523us/step - loss: 0.0493 - acc: 0.9849
Epoch 3/5
60000/60000 [==============================] - 31s 525us/step - loss: 0.0336 - acc: 0.9897
Epoch 4/5
60000/60000 [==============================] - 31s 514us/step - loss: 0.0260 - acc: 0.9920
Epoch 5/5
60000/60000 [==============================] - 32s 526us/step - loss: 0.0189 - acc: 0.99430s - loss: 0.0190 - acc: 0.9


### 모델 평가
- 99.2%의 정확도를 가진다.(기본 데이터를 완전 연결 네트워크(Dense) 보다 합성곱 연산을 한 결과를 완전 연결 네트워크(Dense) 하는것이 더 큰 정확도를 가진다,)

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 1s 136us/step


0.9916